In [1]:
# ML_in_Finance-Viterbi
# Author: Matthew Dixon
# Version: 1.0 (24.7.2019)
# License: MIT
# Email: matthew.dixon@iit.edu
# Notes: tested on Mac OS X with Python 3.6 and Tensorflow 1.3.0
# Citation: Please cite the following reference if this notebook is used for research purposes:
# Dixon M.F., I. Halperin and P. Bilokon, Machine Learning in Finance: From Theory to Practice, Springer Graduate textbook Series, 2020. 

# Overview
This notebook provides a simple example of the Viterbi algorithm applied to an coin which is either fair or loaded (hidden states). Based on a sequence of observations, the algorithm will determine the most likely sequence of hidden states, i.e. whether the coin that generated the data was likely fair or loaded. The example easily maps onto financial markets, e.g. Bull or Bear, Normal or Dislocated etc.

In [32]:
def viterbi(sequence_of_observations, states, initial_probabilities, transition_probabilities, emission_probabilities):
    V = [{}]
    for s in states:
        V[0][s] = {
            'probability': initial_probabilities[s] * emission_probabilities[s][sequence_of_observations[0]],
            'previous': None}
    for t in range(1, len(sequence_of_observations)):
        V.append({})
        for s in states:
            max_tr_probability = V[t-1][states[0]]['probability'] * transition_probabilities[states[0]][s]
            previous_s_selected = states[0]
            for previous_s in states[1:]:
                tr_probability = V[t-1][previous_s]['probability'] * transition_probabilities[previous_s][s]
                if tr_probability > max_tr_probability:
                    max_tr_probability = tr_probability
                    previous_s_selected = previous_s        
            max_probability = max_tr_probability * emission_probabilities[s][sequence_of_observations[t]]
            V[t][s] = {'probability': max_probability, 'previous': previous_s_selected}                    
    most_likely_states = []
    max_probability = max(value['probability'] for value in V[-1].values())
    previous = None
    for s, data in V[-1].items():
        if data['probability'] == max_probability:
            most_likely_states.append(s)
            previous = s
            break
    for t in range(len(V) - 2, -1, -1):
        most_likely_states.insert(0, V[t + 1][previous]['previous'])
        previous = V[t + 1][previous]['previous']
    return {'steps': most_likely_states, 'max_probability': max_probability}

In [33]:
sequence_of_observations = ['Heads', 'Tails', 'Tails', 'Heads', 'Tails', 'Heads', 'Heads', 'Heads', 'Tails', 'Heads']
states = ['Fair', 'Loaded']
initial_probabilities = {'Fair': 0.6, 'Loaded': 0.4}
transition_probabilities = {
    'Fair': {'Fair': 0.6, 'Loaded': 0.4},
    'Loaded': {'Fair': 0.4, 'Loaded': 0.6}
}
emission_probabilities = {
    'Fair': {'Heads': 0.5, 'Tails': 0.5},
    'Loaded': {'Heads': 0.8, 'Tails': 0.2}
}

In [34]:
viterbi(sequence_of_observations,
        states,
        initial_probabilities,
        transition_probabilities,
        emission_probabilities)

{'max_probability': 1.146617856e-05,
 'steps': ['Fair',
  'Fair',
  'Fair',
  'Fair',
  'Fair',
  'Loaded',
  'Loaded',
  'Loaded',
  'Fair',
  'Loaded']}